In [1]:
import time
import torch.optim
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from datasets.dataset import train_loader
import matplotlib.pyplot as plt

C:\Users\Vincent\Desktop\HTW\6. Semester\AkI\cnn-fine-grained-image-recognition\datasets\dataset.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filepath) as f:
C:\Users\Vincent\Desktop\HTW\6. Semester\AkI\cnn-fine-grained-image-recognition\datasets\dataset.py:39: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  x.append(f[name][file].value)


KeyboardInterrupt: 

In [ ]:
learning_rate = 1e-3
input_channels = 1
output_features = 6
epoch = 1
save_model_name = 'convolutional.pth'

In [ ]:
conv = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=True)
conv.eval()
conv.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
conv.fc = nn.Linear(in_features=512, out_features=output_features, bias=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv.parameters(), lr=learning_rate)

In [ ]:
epoch_loss = list()

for epoch_number in range(epoch):
    running_loss, count, acc = 0., 0, 0.
    print(time.asctime())
    for data in train_loader:
        img, label = data
        img = Variable(img)
        label = Variable(label)
        output = conv(img)
        optimizer.zero_grad()
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc += (torch.max(output, dim=1)[1]==label).sum()
        count += img.size(0)
    print(epoch_number, count, running_loss, int(acc)/count)
    epoch_loss.append((count, running_loss, int(acc)/count))

In [ ]:
epoch_loss = list()

In [ ]:
epoch_loss.append((4,6,8))

In [ ]:
plt.plot(range(epoch), epoch_loss)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
torch.save(conv, save_model_name)

In [ ]:
count, acc = 0, 0.
for data in test_loader:
    img, label = data
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    output = conv(img)
    acc += (torch.max(output, dim=1)[1] == label).sum()
    count += img.size(0)
print(count, running_loss, int(acc)/count)